## 이 파일에는 발표할 내용을 적어주세요
- 전처리 내용은 markdown으로 간단히 적고, 질문이 들어올 경우 01_EDA 파일을 보여주는 방식!
- read_csv부터 시작
- 연구배경, 선행연구, 문제정의, 분석방법, 분석결과, 결론 및 앞으로 보완할 점도 들어가는 발표 자료파일 입니다!
- 여기에는 발표할때 보여줄 수있는 가장 깔끔한 형태의 코드만 올려주세요 ㅠㅠ 코딩하는 과정에서 확인하기 위해 하는 과정이나, 발표하기에 부적합한 내용은 다른 파일 또는 폴더를 만들어서 해주세요!
- 다른사람이 작성한 내용은 되도록 수정하거나 삭제하지 말아주세요! 충돌이 일어나요